In [1]:
import sys, os
sys.path.append(os.path.abspath('src'))

from pgai.vector_db_clients import FAISSClient#, ChromaClient
from pgai import langchain_helper

# Intro

In this example, I am using LangChain to build a simple LLM-powered assistant. I will focus on using:
1. Google Gemini
2. OpenAI gpt-4o-mini

# Problem Setting

I'll take a `new_report.txt`, `metrics.csv`, and `training_examples.csv` and use LangChain to:
1. Score the new report
2. Extract from the new_report 3 pieces of supporting evidence for this Score

# Initialize LLM

First need to initialize an LLM. I'll use _Google Gemini_ or _OpenAI 'gpt-4.1-nano'_ model. You can get an API key from [OpenAI](https://platform.openai.com/settings/organization/api-keys).

In [2]:
AI_Provider = "GOOGLE"
# LLM_Provider = "OPENAI"

llm, llm_structured = langchain_helper.get_llm(AI_Provider)

✅ GOOGLE_API_KEY loaded successfully (not printing it for security).


# Create VectorDB from TXT

In [3]:
db_client = FAISSClient()
docs = langchain_helper.split_txt_to_docs(txt_path="data/new_report_Amazon.txt")
db_client.from_docs(docs)
# db = langchain_helper.create_vectordb_from_txt_file(txt_filename="new_report_IBM.txt")
# db.save_local("data/vector_dbs/faiss_index")
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # This model is small (~80MB), fast on CPU, good for English # Alternative: ultra-fast memory-light (~45MB): model_name="sentence-transformers/paraphrase-MiniLM-L3-v2" # Alternative: embeddings = OpenAIEmbeddings()
# db = FAISS.load_local("data/vector_dbs/faiss_index", embeddings)

Report loaded: 303115 characters
First 200 characters of the report: 2024
Amazon
Sustainability
Report
2
Contents
Overview 3 Introduction
4 A Letter from Our
Chief Sustainability
Officer
5 How We Work
6 Goals Summary
7 2024 Year in Review
Progress 8 Carbon and Energy
1...
Split into 1451 chunks
FAISS index has 1451 vectors


# Prompt

In [4]:
import pandas as pd
metrics_filename = 'data/metrics.csv'
metrics = pd.read_csv(metrics_filename)
# print(metrics.iloc[0]["MetricDescription"])
# metrics.head()

In [5]:
train_examples_filename = 'data/train_examples.csv'
train_examples = pd.read_csv(train_examples_filename)
# train_examples.head(1)

In [6]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template('You are a sustainability consultant tasked to score a company against the provided metric. Score can be: 1, 2, or 3.')

# the user prompt is provided by the user, in this case however the only dynamic input is the article
user_prompt = HumanMessagePromptTemplate.from_template(
    """# You need to score the company "{new_company}" against the metric and criteria (provided below) and provide 3 reasons for the score by quoting the report.
        ## The output should be a JSON object with the following fields (no other explanation or text or fields are allowed):
        - Company: the name of the company
        - MetricID: MetricID
        - Score: the score of the company
        - Reason1: first reason for the score
        - Reason2: second reason for the score
        - Reason3: third reason for the score
    
    MetricID: {metric_id}
    Scoring criteria: {metric_description}

    # Below are examples of the scoring applied to 3 companies:
    Company 1: {Company_1}
    Score: {Score_1}
    Reason 1: {Reason1_1}
    Reason 2: {Reason2_1}
    Reason 3: {Reason3_1}

    Company 2: {Company_2}
    Score: {Score_2}
    Reason 1: {Reason1_2}
    Reason 2: {Reason2_2}
    Reason 3: {Reason3_2}

    Company 3: {Company_3}
    Score: {Score_3}
    Reason 1: {Reason1_3}
    Reason 2: {Reason2_3}
    Reason 3: {Reason3_3}
    
    # The report of the company "{new_company}" is:
    {new_company_report_chunks_summary}
    """,

    input_variables=["metric_id", "metric_description", "new_company",
        "Company_1", "Score_1", "Reason1_1", "Reason2_1", "Reason3_1",
        "Company_2", "Score_2", "Reason1_2", "Reason2_2", "Reason3_2",
        "Company_3", "Score_3", "Reason1_3", "Reason2_3", "Reason3_3", "new_company_report_chunks_summary"]
)

Now we can merge the `system` and `user` prompts into a full chat prompt using the `ChatPromptTemplate`:

In [7]:
# prompt template 1: create an article title
prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

# Main Pipeline

Now chain the `prompt` template and the `llm` object defined earlier to create an LLM chain for **prompt formatting > llm generation > get output**.

Let's use __LCEL__ to construct the chain: define inputs with `{"metric_id": lambda x: x["metric_id"], ...}` and use the pipe operator (`|`) to feed the output from its left into the input to its right.

In [8]:
# This chain will output (for the given metric) the new company's score and provides 3 reasons
chain = (
    {
        "metric_id": lambda x: x["metric_id"],
        "metric_description": lambda x: x["metric_description"],
        "new_company": lambda x: x["new_company"],
        "Company_1": lambda x: x["Company_1"],
        "Score_1": lambda x: x["Score_1"],
        "Reason1_1": lambda x: x["Reason1_1"],
        "Reason2_1": lambda x: x["Reason2_1"],
        "Reason3_1": lambda x: x["Reason3_1"],
        "Company_2": lambda x: x["Company_2"],
        "Score_2": lambda x: x["Score_2"],
        "Reason1_2": lambda x: x["Reason1_2"],
        "Reason2_2": lambda x: x["Reason2_2"],
        "Reason3_2": lambda x: x["Reason3_2"],
        "Company_3": lambda x: x["Company_3"],
        "Score_3": lambda x: x["Score_3"],
        "Reason1_3": lambda x: x["Reason1_3"],
        "Reason2_3": lambda x: x["Reason2_3"],
        "Reason3_3": lambda x: x["Reason3_3"],
        "new_company_report_chunks_summary": lambda x: x["new_company_report_chunks_summary"]
    }
    | prompt
    | llm_structured
    | {
        "Company": lambda llm_output: llm_output.Company,
        "MetricID": lambda llm_output: llm_output.MetricID,
        "Score": lambda llm_output: llm_output.Score,
        "Reason1": lambda llm_output: llm_output.Reason1,
        "Reason2": lambda llm_output: llm_output.Reason2,
        "Reason3": lambda llm_output: llm_output.Reason3
        }
)

In [10]:
company_name = "Amazon"
db = langchain_helper.create_vectordb_from_txt_file(
    txt_filename=f"data/new_report_{company_name}.txt"
)

✅ Report loaded (303115 characters).
Sample:
2024
Amazon
Sustainability
Report
2
Contents
Overview 3 Introduction
4 A Letter from Our
Chief Sustainability
Officer
5 How We Work
6 Goals Summary
7 2024 Year in Review
Progress 8 Carbon and Energy
1...
✅ Document split into 1451 chunks.
✅ Vector DB created with 1451 vectors.


In [11]:
new_company_scores = []
company_name = "Amazon"
# Main Scoring Pipeline
def run_scoring_pipeline(company_name, metrics, train_examples, db, chain, top_k=3):
    results = []
    
    for metric_id in metrics.MetricID:
        query = langchain_helper.prep_similarity_search_query(metrics, metric_id)
        report_summary = langchain_helper.similarity_search(query, db, top_k=top_k)
        prompt_inputs = langchain_helper.prep_prompt_inputs(company_name, metrics, metric_id, train_examples, report_summary)
        result = chain.invoke(prompt_inputs)
        results.append(result)

    results_df = pd.DataFrame(results)
    output_path = f"data/{company_name}_scores.csv"
    results_df.to_csv(output_path, index=False)
    print(f"✅ Scoring complete. Results saved to '{output_path}'.")
    return results_df


results_df = run_scoring_pipeline(company_name, metrics, train_examples, db, chain)

✅ Scoring complete. Results saved to 'data/Amazon_scores.csv'.


In [12]:
results_df

,Company,MetricID,Score,Reason1,Reason2,Reason3
0,Amazon,11,3,"Through The Climate Pledge , which we co-found...",our goal to reach net-zero carbon emissions ac...,"We set bold, long-term aspirations, such as Th..."
1,Amazon,12,2,Amazon aims to reach net-zero carbon emissions...,Amazon has committed to The Climate Pledge in ...,Amazon measures both absolute emissions
2,Amazon,13,3,Amazon has set a net-zero carbon emissions tar...,"Through The Climate Pledge, their goal is to r...",They measure and publicly report their carbon ...
3,Amazon,5,1,Scope 2 and 3 carbon emissions are calculated ...,This table includes both on-site solar and con...,We worked with our top suppliers on carbon mea...
4,Amazon,7,1,The report does not provide any information ab...,The report does not provide any information ab...,The report does not provide any information ab...


## <span style="color:red"> __NEXT__ </span>